In [ ]:
pip install --upgrade openai



   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.3 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   -------------------- ------------------- 1.0/2.0 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.4 MB/s  0:00:00

   ---------------------------------------- 0/8 [typing-inspection]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ---------------------------------- 1/8 [tqdm]
   ----- ------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Q1: At what point in the code is the user’s input appended to the conversation history?

The user’s input is appended immediately after it is entered using the line: 
- history.append(f"User: {user_msg}") 
- This ensures that each message is stored in the conversation history before sending the prompt 
to GPT-5.

### Q2: What method is used to send the conversation prompt to GPT-5? 

- The method used is: 
- client.responses.create() 
- This sends the entire conversation history, along with the system instruction, to GPT-5 and 
returns the assistant’s response. 


### Q3: How does the code handle empty user inputs? 
Empty inputs are skipped using: 
- if not user_msg: 
- continue 


This prevents the program from sending blank messages and ensures the chatbot only 
processes actual user input. 

In [ ]:
# Requires: openai >= 1.60.0
# Run: pip install --upgrade openai

from openai import OpenAI

# Create an OpenAI client 
client = OpenAI(api_key="ab93ee06-15de-4c68-9f80-b8c20cbd20dc")

# Conversation state
# Start with a "system" instruction as a *string* (not a list)

# Q4: Modify the code so that the assistant’s replies are limited to at least 15 words and at most 50 words.
system_prompt = ("You are an intelligent assistant. "
    "Ensure your replies are between 15 and 50 words.")

# Q5: Update the script to allow the user to save the entire conversation history into a .txt file once they type save.

# Conversation history
history = [f"System: {system_prompt}"]

print("Type your messages. Type 'bye' to exit, 'save' to save chat, or 'summary' for a summary.\n")

while True:
    user_msg = input("User: ").strip()
    if not user_msg:
        continue

    if user_msg.lower() == "bye":
        print("GPT-5: Goodbye")
        break

    if user_msg.lower() == "save":
        with open("chat_history.txt", "w") as f:
            f.write("\n".join(history))
        print("Conversation saved to chat_history.txt\n")
        continue
###7: Add functionality for the assistant to summarise the conversation so far if the user types summary.  
    if user_msg.lower() == "summary":
        summary_prompt = "\n".join(history) + "\nAssistant: Summarize the conversation briefly."
        response = client.responses.create(model="gpt-5", input=summary_prompt)
        print(f"Summary:\n{response.output_text.strip()}\n")
        continue

# Interactive loop
print("Type your messages. Type 'bye' to exit.\n")

while True:
    user_msg = input("User: ").strip()
    if not user_msg:
        continue
    if user_msg.lower() == "bye":
        print("GPT-5: Goodbye 👋")
        break

 # Save user input
    history.append(f"User: {user_msg}")

    # Combine conversation
    prompt = "\n".join(history)

    
    try:
        # Send the prompt to GPT-5
        response = client.responses.create(
            model="gpt-5",
            input=prompt,
            reasoning={"effort": "medium"},  # optional
            text={"verbosity": "medium"}     # optional
        )

        # Get the assistant’s text
        reply = response.output_text.strip()

        print(f"GPT-5: {reply}\n")

        # Save assistant reply for context in the next turn
        history.append(f"Assistant: {reply}")

    except Exception as e:
        print("Error:", e)
        break
## Q6: Add error handling so that if the API call fails, the program retries three times before quitting.
     # Retry API call up to 3 times
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = client.responses.create(
                model="gpt-5",
                input=prompt,
                reasoning={"effort": "medium"},
                text={"verbosity": "medium"}
            )

            reply = response.output_text.strip()
            print(f"GPT-5: {reply}\n")

            # Save assistant reply
            history.append(f"Assistant: {reply}")
            break

        except Exception as e:
            print(f"API call failed (attempt {attempt+1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("Giving up after 3 failed attempts.\n")


Type your messages. Type 'bye' to exit.

GPT-5: Goodbye 👋
